In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle


In [2]:
## load the trained model, pickel file
model = load_model('model.h5')

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geography.pkl', 'rb') as file:
    one_hot_encoder_geography = pickle.load(file)


In [3]:
# Example input data
input_data = {
    'CreditScore': 500,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 30,
    'Tenure': 2,
    'Balance':40000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary':30000
}

In [4]:
## one hot encoded  geography
geo_encoded = one_hot_encoder_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geography.get_feature_names_out(['Geography']))
geo_encoded_df


c:\Users\hp\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,France,Male,30,2,40000,2,1,1,30000


In [6]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,France,1,30,2,40000,2,1,1,30000


In [7]:
## concatination one hot encoded 
input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df = input_df.drop(columns=['Geography'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,500,1,30,2,40000,2,1,1,30000,1.0,0.0,0.0


In [8]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-1.57375827,  0.91324755, -0.84593077, -1.04241787, -0.57803098,
         0.80843615,  0.64920267,  0.97481699, -1.22456561,  1.00150113,
        -0.57946723, -0.57638802]])

In [9]:
## Predicting the output
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step


array([[0.00751302]], dtype=float32)

In [10]:
predection_probability = prediction[0][0]
predection_probability

0.0075130234

In [11]:
if predection_probability > 0.5:
    print('The customer is likely to exit')
else:
    print('The customer is not likely to exit')

The customer is not likely to exit
